# GOAL : Scrape Drama Reviews
FROM: https://mydramalist.com/

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
from selenium.common.exceptions import NoSuchElementException      

In [2]:
driver_place = 'C:/Users/Hanna Kim/Documents/데잇걸즈 4기/공공데이터 공모전/chromedriver'

#고정된 driver page열어주기
driver = webdriver.Chrome(driver_place)

### 반복되는 코드 함수로 만들기

In [3]:
def get_frame(soup):
    title = []
    reviewer = []
    # create list for each column
    story = []
    acting = []
    music = []
    rewatch = []
    tire = soup.select("a.text-primary")
    score = soup.select("span.p-l-md.pull-right")
    for i in range(int(len(tire)/3)):
        title.append(tire[3*i].text)
        reviewer.append(tire[3*i+1].text)
        story.append(score[4*i].text) 
        acting.append(score[4*i+1].text)
        music.append(score[4*i+2].text)
        rewatch.append(score[4*i+3].text)
    overall = [*map(lambda x:x.text,soup.select("span.score"))]
    
    return pd.DataFrame({"title":title, "reviewer":reviewer, "overall":overall,"story":story,"acting":acting, "music":music, "rewatch":rewatch})


def drama_users(drama):
    '''
    get top 12 reviewers in drama
    
    try with example:
        18452-goblin
        18894-strong-woman-do-bong-soon
    
    drama_users("drama tag for link")
    '''
    url = f"https://mydramalist.com/{drama}/reviews"
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    prof = []
    for i in soup.select("a.text-primary"):
        prof.append(i.get("href"))
    profile_list = [x[9:] for x in prof if x.startswith("/profile") & ~x.endswith("/reviews")][4:]
    return profile_list


def check_exist_next():
    '''
    #when exists
    url = 'https://mydramalist.com/profile/dragynfaerie/reviews'
    driver.get(url)
    check_exist_class()

    #when not
    url = 'https://mydramalist.com/profile/Chacha35/reviews'
    driver.get(url)
    check_exist_class()
    '''
    try:
        driver.find_element_by_class_name("page-item.nb.active")
    except NoSuchElementException:
        return False
    return True

### 최종 구성

In [13]:

# profile_list = ['BounamaHoussam', 'dragynfaerie','Tanya94','Chacha35']
# 테스트용 프로필 리스트 원래는 drama_users로 구해야함


profile_list = drama_users('18452-goblin')
many_users = pd.DataFrame({"title":[], "reviewer":[], "overall":[],"story":[],"acting":[], "music":[], "rewatch":[]})

for user_id in profile_list:
    
    url = f'https://mydramalist.com/profile/{user_id}/reviews'
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(10)
    # review 없는 경우 제외
    if soup.select("div.box-body")[-1].text == "There have been no reviews submitted.":
        print("this has no reviews")
    
    else:
        print("this has reviews")
        
        #첫번째 페이지 긁어서 프레임 만들기
        page_num = 1
        pages_frame = get_frame(soup)
        
        # 리뷰가 12개 미만이면 더 구할게 없음
        if check_exist_next(): #when next page exists
            while soup.select_one("li.page-item.nb.active").text == str(page_num):
                if soup.select_one("li.page-item.nb.active").text != str(page_num): 
                    break
                page_num += 1
                page_url = url + f'?page={page_num}'
                driver.get(page_url)
                html = driver.page_source
                soup = BeautifulSoup(html, 'html.parser')
                time.sleep(10)
                imsi =get_frame(soup)
                pages_frame = pd.concat([pages_frame, imsi])
            many_users = pd.concat([many_users, pages_frame])
        else:
            many_users = pd.concat([many_users, pages_frame])

        

this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews
this has reviews


In [14]:
many_users

,title,reviewer,overall,story,acting,music,rewatch
0,Mr. Sunshine,kingsqueen,10,10,10,10,10
1,Goblin,kingsqueen,10,10,10,10,10
2,The Bride of Habaek,kingsqueen,7.0,7.0,7.0,9.0,5.5
3,Jealousy Incarnate,kingsqueen,8.5,8.5,10,10,9.0
4,Suspicious Partner,kingsqueen,7.0,7.0,9.0,7.0,5.0
...,...,...,...,...,...,...,...
3,One More Time,Ivana,7.5,5.5,7.5,9.5,7.5
4,The Village: Achiara's Secret,Ivana,9.5,9.0,10,10,7.5
5,The Guardians,Ivana,10,10,10,10,10
0,W,VandaLimbata,5.5,5.0,6.5,9.0,1.0


In [75]:
profile_list

['kingsqueen75',
 'dragynfaerie',
 'Ghettoe',
 'Tanya94',
 'wonhwa',
 'Mrmz',
 'sambart',
 'Chacha35',
 'Nathansgurl',
 'Karoline00',
 'Ive',
 'VandaLimbata']

## turn into class

In [22]:
class PerDrama():
    
    def __init__(self, drama):
        self.drama = drama
    
    def run(self):
        profile_list = drama_users(self.drama)
        many_users = pd.DataFrame({"title":[], "reviewer":[], "overall":[],"story":[],"acting":[], "music":[], "rewatch":[]})

        for user_id in profile_list:

            url = f'https://mydramalist.com/profile/{user_id}/reviews'
            driver.get(url)
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            time.sleep(10)
            print("got user")

            # review 없는 경우 제외
            
            if soup.select("div.box-body")[-1].text == "There have been no reviews submitted.":
                print("this has no reviews")
                
            else:
                print("this has reviews")

                #첫번째 페이지 긁어서 프레임 만들기
                page_num = 1
                pages_frame = get_frame(soup)

                # 리뷰가 12개 미만이면 더 구할게 없음
                if check_exist_next(): #when next page exists
                    while soup.select_one("li.page-item.nb.active").text == str(page_num):
                        if soup.select_one("li.page-item.nb.active").text != str(page_num): 
                            break
                        page_num += 1
                        page_url = url + f'?page={page_num}'
                        driver.get(page_url)
                        html = driver.page_source
                        soup = BeautifulSoup(html, 'html.parser')
                        time.sleep(10)
                        imsi =get_frame(soup)
                        pages_frame = pd.concat([pages_frame, imsi])
                    many_users = pd.concat([many_users, pages_frame])
                else:
                    many_users = pd.concat([many_users, pages_frame])

        return many_users

In [86]:
many_users

,title,reviewer,overall,story,acting,music,rewatch
0,Mr. Sunshine,kingsqueen,10,10,10,10,10
1,Goblin,kingsqueen,10,10,10,10,10
2,The Bride of Habaek,kingsqueen,7.0,7.0,7.0,9.0,5.5
3,Jealousy Incarnate,kingsqueen,8.5,8.5,10,10,9.0
4,Suspicious Partner,kingsqueen,7.0,7.0,9.0,7.0,5.0
...,...,...,...,...,...,...,...
3,One More Time,Ivana,7.5,5.5,7.5,9.5,7.5
4,The Village: Achiara's Secret,Ivana,9.5,9.0,10,10,7.5
5,The Guardians,Ivana,10,10,10,10,10
0,W,VandaLimbata,5.5,5.0,6.5,9.0,1.0


In [23]:
PerDrama('/18452-goblin').run()

got user
this has reviews


,title,reviewer,overall,story,acting,music,rewatch
0,Mr. Sunshine,kingsqueen,10,10,10,10,10
1,Goblin,kingsqueen,10,10,10,10,10
2,The Bride of Habaek,kingsqueen,7.0,7.0,7.0,9.0,5.5
3,Jealousy Incarnate,kingsqueen,8.5,8.5,10,10,9.0
4,Suspicious Partner,kingsqueen,7.0,7.0,9.0,7.0,5.0
...,...,...,...,...,...,...,...
1,Backstreet Rookie,kingsqueen,7.5,7.0,7.5,9.0,7.0
2,My Little Bride,kingsqueen,8.0,9.0,9.0,5.0,8.0
3,My P.S. Partner,kingsqueen,8.0,8.0,9.0,8.0,6.0
4,Always,kingsqueen,9.5,10,9.5,7.5,9.5


In [26]:
#test용으로 두 드라마 리뷰의 첫 유저들로만 해봄
drama_list = ['/16589-w','/18452-goblin' ]
many_dramas = pd.DataFrame({"title":[], "reviewer":[], "overall":[],"story":[],"acting":[], "music":[], "rewatch":[]})
for drama in drama_list:
    imsi = PerDrama(drama).run()
    many_dramas = pd.concat([many_dramas, imsi])

got user
this has reviews
got user
this has reviews


In [27]:
many_dramas

,title,reviewer,overall,story,acting,music,rewatch
0,My Absolute Boyfriend,Richel,6.0,4.5,7.0,5.0,1.0
1,Hotel del Luna,Richel,8.0,6.0,9.0,10,5.0
2,Skate Into Love,Richel,8.0,8.0,9.0,8.0,7.5
3,A Love So Beautiful,Richel,8.5,7.5,10,8.0,7.5
4,Long For You,Richel,4.5,3.0,6.0,7.5,1.0
...,...,...,...,...,...,...,...
1,Backstreet Rookie,kingsqueen,7.5,7.0,7.5,9.0,7.0
2,My Little Bride,kingsqueen,8.0,9.0,9.0,5.0,8.0
3,My P.S. Partner,kingsqueen,8.0,8.0,9.0,8.0,6.0
4,Always,kingsqueen,9.5,10,9.5,7.5,9.5


## top drama links

In [28]:
driver.get("https://mydramalist.com/shows/popular")
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [29]:
list_with_none = [x.get("href") for x in soup.select('h6.text-primary.title>a')]

In [30]:
list_with_none

['/18452-goblin',
 None,
 '/18894-strong-woman-do-bong-soon',
 None,
 '/2987-boys-over-flowers',
 None,
 '/10904-descendants-of-the-sun',
 None,
 '/7398-you-who-came-from-the-stars',
 None,
 '/6640-the-heirs',
 None,
 '/16589-w',
 None,
 '/19262-weightlifting-fairy-kim-bok-joo',
 None,
 '/11074-pinocchio',
 None,
 '/10814-healer',
 None,
 '/8-secret-garden-2010',
 None,
 '/21576-while-you-were-sleeping',
 None,
 '/27885-why-secretary-kim',
 None,
 '/2986-youre-beautiful',
 None,
 '/15999-moon-lovers-scarlet-heart-ryeo',
 None,
 '/10873-kill-me-heal-me',
 None,
 '/2982-coffee-prince',
 None,
 '/1702-city-hunter-2011',
 None,
 '/6993-i-hear-your-voice',
 None,
 '/18816-the-legend-of-the-blue-sea',
 None]

In [31]:
famous_drama = [i for i in list_with_none if i]
famous_drama

['/18452-goblin',
 '/18894-strong-woman-do-bong-soon',
 '/2987-boys-over-flowers',
 '/10904-descendants-of-the-sun',
 '/7398-you-who-came-from-the-stars',
 '/6640-the-heirs',
 '/16589-w',
 '/19262-weightlifting-fairy-kim-bok-joo',
 '/11074-pinocchio',
 '/10814-healer',
 '/8-secret-garden-2010',
 '/21576-while-you-were-sleeping',
 '/27885-why-secretary-kim',
 '/2986-youre-beautiful',
 '/15999-moon-lovers-scarlet-heart-ryeo',
 '/10873-kill-me-heal-me',
 '/2982-coffee-prince',
 '/1702-city-hunter-2011',
 '/6993-i-hear-your-voice',
 '/18816-the-legend-of-the-blue-sea']

In [111]:
many_dramas = pd.DataFrame({"title":[], "reviewer":[], "overall":[],"story":[],"acting":[], "music":[], "rewatch":[]})
for drama in famous_drama:
    imsi = PerDrama(drama).run()
    many_dramas = pd.concat([many_dramas, imsi])

['BounamaHoussam',
 'hideyourheart',
 'Hessa',
 'hyungyshik',
 'kingsqueen75',
 'pinkspades',
 'manicmuse',
 'Thevilone',
 'Tsukkitori',
 'lovecoutureflyy',
 'Wererabbit',
 '_Rosie',
 'HermanFassett',
 'vale383',
 'binja-man697',
 'MichaelTang']